<a href="https://colab.research.google.com/github/Pallavi-Bhaskar/corsera-test/blob/main/AnimeRecommendatiosAtUr1Click.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip  install -q kaggle

In [14]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"paulsi","key":"ea224f4d190cef972d1bebe0f20082a1"}'}

In [15]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [16]:
! cp kaggle.json ~/.kaggle/

In [17]:
! chmod 600 ~/.kaggle/kaggle.json

In [18]:
! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                        Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23          12549        369  1.0              
zusmani/petrolgas-prices-worldwide                              Petrol/Gas Prices Worldwide                          10KB  2022-06-24 01:25:33           1144         71  1.0              
devansodariya/student-performance-data                          Student Performance Dataset                           7KB  2022-05-26 13:55:09           7263        209  0.9705882        
ruchi798/data-science-job-salaries                          

In [22]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

 98% 649M/661M [00:05<00:00, 128MB/s] 
100% 661M/661M [00:06<00:00, 115MB/s]


In [23]:
!unzip anime-recommendation-database-2020.zip

Archive:  anime-recommendation-database-2020.zip
  inflating: anime.csv               
  inflating: anime_with_synopsis.csv  
  inflating: animelist.csv           
  inflating: html folder/html/1/details.html  
  inflating: html folder/html/1/pictures.html  
  inflating: html folder/html/1/recomendations.html  
  inflating: html folder/html/1/reviews_1.html  
  inflating: html folder/html/1/reviews_10.html  
  inflating: html folder/html/1/reviews_11.html  
  inflating: html folder/html/1/reviews_12.html  
  inflating: html folder/html/1/reviews_13.html  
  inflating: html folder/html/1/reviews_14.html  
  inflating: html folder/html/1/reviews_15.html  
  inflating: html folder/html/1/reviews_16.html  
  inflating: html folder/html/1/reviews_17.html  
  inflating: html folder/html/1/reviews_18.html  
  inflating: html folder/html/1/reviews_19.html  
  inflating: html folder/html/1/reviews_2.html  
  inflating: html folder/html/1/reviews_20.html  
  inflating: html folder/html/1/reviews

#Libraries

In [24]:
import os
import numpy as np
import pandas as pd

import warnings

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

#Importing data

In [25]:
anime_synop_df = pd.read_csv('anime_with_synopsis.csv')
anime_complete_df = pd.read_csv('anime.csv')

In [52]:
anime_synop_df.head(10)

,MAL_ID,Name,Genres,sypnopsis
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
5,15,Eyeshield 21,"Action, Sports, Comedy, Shounen",Sena is like any other shy kid starting high s...
6,16,Hachimitsu to Clover,"Comedy, Drama, Josei, Romance, Slice of Life","Yuuta Takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,"Slice of Life, Comedy, Sports, Shounen",Kyosuke Kano has lived under the shadow of his...
8,18,Initial D Fourth Stage,"Action, Cars, Sports, Drama, Seinen",Takumi Fujiwara finally joins Ryousuke and Kei...
9,19,Monster,"Drama, Horror, Mystery, Police, Psychological,...","Dr. Kenzou Tenma, an elite neurosurgeon recent..."


In [27]:
anime_synop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


In [28]:
anime_synop_df = anime_synop_df[['MAL_ID', 'Name', 'Genres', 'Synopsis']]

In [29]:
anime_df = anime_synop_df.join(anime_complete_df, on='MAL_ID', rsuffix='r')

In [30]:
anime_df = anime_df[['Name', 'Genres', 'Sypnopsis', 'Type']]

In [31]:
anime_df.columns = ['Name', 'Genres', 'Synopsis', 'Type']

In [56]:
anime_df

,Name,Genres,Synopsis
1,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
5,Eyeshield 21,"Action, Sports, Comedy, Shounen",Sena is like any other shy kid starting high s...
...,...,...,...
16209,Daomu Biji Zhi Qinling Shen Shu,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,Mieruko-chan,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,Yama no Susume: Next Summit,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


#Missing Values

In [33]:
anime_df.isnull().sum().sort_values(ascending=False)

Type        9436
Synopsis       8
Name           0
Genres         0
dtype: int64

In [34]:
print(anime_df['Synopsis'].mode()[0])
print(anime_df['Type'].mode()[0])

No synopsis information has been added to this title. Help improve our database by adding a synopsis here .
TV


In [35]:
anime_df['Synopsis'] = anime_df['Synopsis'].fillna(
    anime_df['Synopsis'].dropna().mode().values[0]
)
anime_df['Type'] = anime_df['Type'].fillna(
    anime_df['Type'].dropna().mode().values[0]
)

In [36]:
anime_df.isnull().sum()

Name        0
Genres      0
Synopsis    0
Type        0
dtype: int64

In [37]:
anime_df = anime_df[anime_df['Type']=='TV']

In [38]:
anime_df.drop('Type', axis=1, inplace=True)

#Building Of Recommender

In [39]:
indices = pd.Series(anime_df.index, index = anime_df['Name'])

In [43]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf2 = TfidfVectorizer(stop_words='english')
count = CountVectorizer(stop_words='english')

In [44]:
tfidf_matrix = tfidf.fit_transform(anime_df['Name'])
count_matrix = count.fit_transform(anime_df['Genres'])
tfidf2_matrix = tfidf2.fit_transform(anime_df['Synopsis'])

In [45]:
name_similarity = cosine_similarity(tfidf_matrix)
genre_similarity = cosine_similarity(count_matrix)
synopsis_similarity = cosine_similarity(tfidf2_matrix)

In [62]:
def get_recommendations(anime):
    i = indices[anime]
    
    name_score = list(enumerate(name_similarity[i]))
    genre_score = list(enumerate(genre_similarity[i]))
    synopsis_score = list(enumerate(synopsis_similarity[i]))
    
    name_score = sorted(name_score, key = lambda x: x[0])
    genre_score = sorted(genre_score, key = lambda x: x[0])
    synopsis_score = sorted(synopsis_score, key = lambda x: x[0])
    
    combined_score = [(i, (sc_1 + sc_2 + sc_3) / 3) for (i, sc_1), (_, sc_2), (_, sc_3) in zip(name_score, genre_score, synopsis_score)]
    
    combined_score = sorted(combined_score, key = lambda x: x[1], reverse = True)
    
    anime_ids = [i[0] for i in combined_score[1:11]]

    anime_recs = []
    
    index = 0
    while len(anime_recs) != 10:
        anime_id = combined_score[1:][index][0]
        index += 1
        if anime in indices.iloc[[anime_id]].index[0]:
            continue
        else:
            anime_recs.append(indices.iloc[[anime_id]].index[0])
    
    
    print(f'Anime similar to {anime}:')
    for i, v in list(enumerate(anime_recs)):
        print(f'{i + 1}. {v}')